In [ ]:
# This has to go in its own cell or it screws up the defaults we'll set later
%matplotlib inline

In [ ]:
import numpy as np
import musictoys
import musictoys.audiofile
import musictoys.analysis
import musictoys.spectral
from scrapbook import plot
import librosa
import librosa.feature
import librosa.feature.spectral

In [ ]:
#filedata, filerate = musictoys.audiofile.read("audio_files/kronfeld-dreamatic.wav")
#filedata, filerate = musictoys.audiofile.read("audio_files/jfb-back_home.wav")
filedata, filerate = musictoys.audiofile.read("audio_files/liberty_chaps-get_up_get_down.wav")
samples, samplerate = musictoys.analysis.normalize(filedata, filerate)

In [ ]:
frame_size = 1024
step_size = frame_size / 2
step_rate = samplerate / step_size
num_frames = int((len(samples) - frame_size + 1) / step_size)

In [ ]:
frames = np.array([samples[i*step_size:i*step_size+frame_size] for i in np.arange(num_frames)])

In [ ]:
hann = np.hanning(frame_size)

spectrogram = []
for clip in frames:
    # weight according to the hann window and take the FFT
    # this version does not return the symmetrical components
    spectrum = np.fft.rfft(clip * hann)
    # add this spectrum to the series of spectra
    spectrogram.append(spectrum)

spectrogram = np.array(spectrogram)
# take absolute value to get magnitude
spectrogram = np.absolute(spectrogram)
# square to get power, normalize by the FFT window size -
# but adjust for the fact that we threw away the mirrored bins.
powerspectrogram = np.square(spectrogram) / (frame_size/2)
# convert to decibels, clip noise floor
loudnessgram = (10 * np.log10(powerspectrogram)).clip(-120)
# compute the center frequency of each FFT bin
nyquist_freq = samplerate / 2.0
fft_bin_freq = np.arange(spectrogram.shape[-1]) * nyquist_freq / float(spectrogram.shape[-1])

In [ ]:
plot.gram(loudnessgram, x=step_rate, y=fft_bin_freq, cmap='plasma')

# CENTROID

In [ ]:
plot.line(musictoys.spectral.centroid(spectrogram, samplerate), x=step_rate)

In [ ]:
plot.line(librosa.feature.spectral_centroid(S=spectrogram.T, sr=samplerate)[0])

# SPREAD/BANDWIDTH

In [ ]:
plot.line(musictoys.spectral.spread(spectrogram, samplerate), x=step_rate)

In [ ]:
plot.line(librosa.feature.spectral_bandwidth(S=spectrogram.T, sr=samplerate)[0])

# VARIANCE

In [ ]:
plot.line(musictoys.spectral.variance(spectrogram), x=step_rate)

# CREST

In [ ]:
plot.line(musictoys.spectral.crest(spectrogram), x=step_rate)

# ENTROPY

In [ ]:
plot.line(musictoys.spectral.entropy(powerspectrogram), x=step_rate)

# FLATNESS

In [ ]:
plot.line(musictoys.spectral.flatness(powerspectrogram), x=step_rate)

# ROLLOFF

In [ ]:
plot.line(musictoys.spectral.rolloff(spectrogram, samplerate, fraction=0.85))

In [ ]:
plot.line(librosa.feature.spectral.spectral_rolloff(S=spectrogram.T, sr=samplerate, roll_percent=.85)[0])

# ZERO-CROSSING RATE

In [ ]:
plot.line(musictoys.analysis.zcr(frames))

worth investigating?
- skewness
- kurtosis
- slope
- decrease
- high-frequency content